- 인공지능은 우리 생활과 일에 지능을 더합니다. 이 지능은 다른 종류의 지능입니다. 여러 분야에서 특히 예술에서는 AI가 사람의 능력을 증가시키는 도구로 사용될 것입니다. 즉 인공적인 지능이 아니라 확장된 지능입니다.


- 예술 창작의 대부분은 간단한 패턴 인식과 기술로 만들어집니다. 많은 사람은 이런 과정에 관심이 없거나 심지어 불필요하다고 생각하기도 합니다. 여기에 AI가 필요합니다. 사람의 지각, 언어 예술 작품은 모두 통계적 구조를 가집니다. 딥러닝 알고리즘은 이 구조를 학습하는 데 뛰어납니다. 머신 러닝 모델은 이미지, 음악, 글의 잠재적 공간을 학습할 수 있습니다.


- 이 장에서 예술 창작에 딥러닝이 어떻게 쓰일 수 있는지 다양한 각도에서 살펴보겠습니다. 시퀀스 데이터 생성과 딥드림, 변이형 오토인코더, 적대적 생성 네트워크를 사용한 이미지 생성을 알아보겠습니다.

# 1. LSTM으로 텍스트 생성하기

- 이 절에서 순환 신경망으로 시퀀스 데이터를 생성하는 방법을 살펴보겠습니다. 텍스트 생성을 에로 들지만 동일한 기법으로 어떤 종류의 시퀀스 데이터도 생성할 수 있습니다. 음표의 시퀀스에 적용하여 새로운 음악을 만들거나 연속된 붓질 시퀀스에 적용하여 한 획 한 획 그림을 그릴 수 있습니다.

- 딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇 개의 토큰을 예측하는 것입니다. 예를 들어 'the cat is on the mat'란 입력이 주어지면 다음 글자인 타깃 't'를 예측하도록 네트워크를 훈련시킵니다. 텍스트 데이터를 다룰 때 토큰은 봍오 단어 또는 글자입니다. 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 언어 모델이라고 부릅니다.

## 1. 샘플링 전략의 중요성

1. 탐욕적 샘플링: 텍스트를 생성할 때 다음 글자를 생성하는 방법이 아주 중요합니다. 단순한 방법은 항상 가장 높은 확률을 가진 글자를 선택하는 방법입니다. 이 방법은 반복적이기 때문에 논리적인 언어처럼 보이지 않는다.


2. 확률적 샘플링: 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법입니다. 이를 확률적 샘플링이라고 부릅니다.
    - 샘플링 과정에서 무작위성의 양을 조절한 방법이 없다.
    
    
3. 소프트맥스 온도: 샘플링 과정에서 확률의 양을 조절하기 위해 소프트맥스 온도라는 파라미터를 사용합니다. 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타냅니다. 얼마나 놀라운 또는 예상되는 글자를 선택할지 결정합니다.

In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

## 2. 글자 수준의 LSTM 텍스트 생성 모델 구현

- 니체의 글을 인용한다.

In [2]:
import keras
import numpy as np

Using TensorFlow backend.


### 1. 데이터

In [3]:
# 원본 텍스트 파일을 내려받아 파싱하기
path = keras.utils.get_file('nietzche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print("말 뭉치 크기: ", len(text))

606208/600901 [==============================] - 1s 1us/step
말 뭉치 크기:  600893


In [4]:
# 글자 시퀀스 벡터화하기
maxlen = 60
step = 3
sentences = []
next_chars = []

# 3글자씩 뛰어넘으면서 중복 추출한다.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [8]:
print("시퀀스 개수: ", len(sentences))

시퀀스 개수:  200278


In [9]:
chars = sorted(list(set(text)))
print("고유한 글자: ", len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

고유한 글자:  57


In [11]:
print("벡터화")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

# 글자를 원핫 인코딩하여 0과 1의 이진 배열로 바꾼다.
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

벡터화


### 2. 네트워크 구성

In [14]:
from keras import layers

# 이 네트워크는 하나의 LSTM 층과 그 뒤에 Dense 분류기가 뒤따른다.
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [15]:
# 타깃이 원핫 인코딩 되어 있기 때문에 모델 훈련을 위해 categorical_crossentropy 손실을 사용한다.
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### 3. 언어 모델 훈련과 샘플링

- 훈련된 모델과 seed로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있다.

1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑는다.
2. 특정 온도로 이 확률 분포의 가중치를 조정한다.
3. 가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4. 새로운 글자를 생성된 텍스트의 끝에 추가한다.

In [16]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

- 마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성한다. 에포크마다 학습이 끝난 후 여러 가지 온도를 사용해서 텍스트를 생성한다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있다. 온도가 샘플링 전략에 미치는 영향도 보여줍니다.

In [17]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)
start_index

116739

In [19]:
for epoch in range(1, 10):
    print("에폭", epoch)
    
    # 데이터에서 한 번만 반복해서 모델을 학습한다.
    model.fit(x, y, batch_size=128, epochs=1)
    
    # 무작위로 시드 텍스트를 선택한다.
    seed_text = text[start_index: start_index + maxlen]
    print("--- 시드 텍스트: '" + seed_text + "'")
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("------ 온도: ", temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        # 시드 텍스트에서 시작해서 400개 글자를 생성한다.
        for i in range(400):
            
            # 생성된 글자를 원핫 인코딩으로 바꾼다.
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            # 다음 글자 샘플링
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에폭 1
Epoch 1/1
200278/200278 [==============================] - 74s 370us/step - loss: 1.3647
--- 시드 텍스트: 'the slowly ascending ranks and classes, in which,
through fo'
------ 온도:  0.2
the slowly ascending ranks and classes, in which,
through forms of the spirit of the same time the spirit and the same the spirit and the strength the spirit of the same the spirit and the senses of the than the same the spiritual proper and the soul, a something all the spirit of the spirit of the spirit and proper and the senses of the spirit of the spirit of the delights of the spirit and so case of the spiritual soul the strength to the senses of the s
------ 온도:  0.5
the slowly ascending ranks and classes, in which,
through former the rasting for example, the sense of the pattic and sacrifice must above all act of a morality of distrust in the chastible everything begins and conscience, of the spiritual proper the laugh the will comes the contrary desirability of the free spirits of the spirit contr

"pule liker, he woukes, actuous, have yone; uprem-thought to as occasionally inatorapsyy, of a deglection, eurous will 
에폭 5
Epoch 1/1
200278/200278 [==============================] - 73s 364us/step - loss: 1.3475
--- 시드 텍스트: 'the slowly ascending ranks and classes, in which,
through fo'
------ 온도:  0.2
the slowly ascending ranks and classes, in which,
through for the same explanation of the spirit of the present may be a more superstition and the spirit of the spirit of the same seems to the same as the same super-itself as the same the spirit of the same the same the same the strength that the same the same the spirit of the same as a more discoverness of the same and the same the spirit of the spirit of the same and the same as the same the same and th
------ 온도:  0.5
the slowly ascending ranks and classes, in which,
through forms of the strength
in the common of the strength of the spirit that it is the desirably and the same talk and form, its assumed to perhaps one of the same th

/home/lifesailor/anaconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 the independencuness, in the subjuction heip of knows
to
에폭 6
Epoch 1/1
200278/200278 [==============================] - 72s 362us/step - loss: 1.3415
--- 시드 텍스트: 'the slowly ascending ranks and classes, in which,
through fo'
------ 온도:  0.2
the slowly ascending ranks and classes, in which,
through for in the conception of the sense of the sensual the same the subject and subjucts the sense of the present the sense of the servilution of the sense of the sense of the sense of the suffering of the suffering the self-every thing to the higher a perceived the sense of the suffering the sense of the sense of the sense of the sense of the reason and the sense of the self-ender of the sense of the se
------ 온도:  0.5
the slowly ascending ranks and classes, in which,
through for like the suffering of a morality of
abjerailing and life in the percesies to have the stomed estimated of the precisely as a man and the induction of the subjuctic as a course, the sense, and of the astempt of the more

exure moments. for an troes--on that mentaugids of it like this certain, intedemano, of renemainable of
power, practoly but i and feels become it, conscies smiat, maniness, fy his
have hithertow!

7béas to wayni


- 낮은 온도는 반복적이고 예상되는 텍스트를 만든다. 하지만 국부적인 구조는 실제와 매우 같다. 특히 모든 단어가 실제 영어 단어이다.

- 높은 온도에서 생성된 텍스트느 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따끔 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다. 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느 정도 무작위한 문자열로 보입니다.텍스트 생성에 가장 좋은 온도는 0.5이다. 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

- 당연히 의미 있는 텍스트가 생성된다고 생각하지 마세요. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어 의미가 줄지는 않지만 고유한 통계 구조가 사라질 것입니다.

## 3. 정리

1. 이전 토큰이 주어지면 다음 토큰을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있다.

2. 텍스트의 경우 이런 모델을 언어 모델이라고 부른다. 단어 또는 글자 단위 모두 가능하다.

3. 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 한다.

4. 이를 위해 소프트맥스 온도 개념을 사용한다. 항상 다양한 온도를 실험해서 적절한 값을 찾는다.